In [64]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotVisibleException, InvalidArgumentException

In [ ]:
# chromedriver = '/usr/local/bin/chromedriver'
# driver = webdriver.Chrome(chromedriver)
# select = driver.find_elements_by_css_selector
# select_one = driver.find_element_by_css_selector
#start linkedIn with a fresh page, no one is logged in
#driver.get('https://www.linkedin.com/')

In [67]:
def sign_in(login, password, driver):#if on the main linked in with the sign in on top right
    select = driver.find_elements_by_css_selector
    select_one = driver.find_element_by_css_selector
    driver.get('https://www.linkedin.com/')
    try:
        sign_in = select_one('input#login-email')
        sign_in.click()
        sign_in.send_keys(login)
        password_in = select_one('input#login-password')
        password_in.click()
        password_in.send_keys(password)
        login = select_one('input#login-submit')
        login.click()
    except NoSuchElementException:       
        try:
            sign_in = select_one('a.nav__button-secondary')
            sign_in.click()
            try:
                input_name = select_one('input#username')
                input_name.send_keys(login)
                input_pwd = select_one('input#password')
                input_pwd.send_keys(password)
                login_action = select_one('div.login__form_action_container ')
                login_action.click()
            except NoSuchElementException:
                    driver.get(sign_in.get_attribute('href'))
                    input_name = select_one('input#username')
                    input_name.click()
                    input_name.send_keys(login)
                    input_password = select_one('input#password')
                    input_password.click()
                    input_password.send_keys(password)
                    login_btn = select_one('button.btn__primary--large.from__button--floating')
                    login_btn.click()
        except NoSuchElementException:
            try:
                email_input = select_one('input.login-email')
                email_input.click()
                email_input.send_keys(input)
                password_input = select_one('input.login-password')
                password_input.click()
                password_input.send_keys(password)
                submit = select_one('#login-submit')
                submit.click()

            except:
                print('nope, didnt work')
    return(driver)


In [27]:
login = 'keri@multerer.com'
password = 'jfdkfjadluivjdkk5'

# Getting list of conversations on the left and then clicking through them to capture the full message thread

In [93]:
def get_messages(driver):
    select = driver.find_elements_by_css_selector
    select_one = driver.find_element_by_css_selector
    df = pd.DataFrame(columns=['id','message','date'])
    #accessing the messages once logged in
    message_btn = select_one('span#messaging-tab-icon')
    message_btn.click()
    try:
        unordered_list = select_one('ul.msg-conversations-container__conversations-list')
        convo_list = unordered_list.find_elements_by_css_selector('li')
        print(len(convo_list))

#         print(unordered_list)
#         url_elements = unordered_list.find_elements_by_css_selector('a')
#         print(len(url_elements))
#         convo_list = []
#         for element in url_elements:
#             convo_list.append(str(element.get_attribute('href')))
#         print(len(convo_list))
#         print(convo_list)
    except NoSuchElementException:
        print('in 1st no such element')
        convo_list = select('li.msg-conversation-listitem')
        print(len(convo_list))
    for convo in convo_list:
        try:
            convo.click()
            #driver.get(convo)
            try:
                message = select_one('div.msg-s-message-list').text
                message_list = message.split('\n')
                time.sleep(1)
                if message_list[0] == 'Recruiter':
                    df = df.append({'id': message_list[0], 'message': message, 'date': message_list[1]}, ignore_index=True)
                elif '2' in message_list[0]:
                    if len(message_list) >= 4:
                        df = df.append({'id': message_list[3], 'message': message, 'date': message_list[0]}, ignore_index=True)
                    else:
                        print(message_list)
                        df = df.append({'id': message_list[1][:-21], 'message': message, 'date': message_list[0]}, ignore_index=True)
                else:
                    df = df.append({'id': message_list[0], 'message': message, 'date': message_list[5]}, ignore_index=True)
            except NoSuchElementException:
                df = df.append({'id': '' , 'message' : 'advertisement', 'date': ''}, ignore_index=True)
        except ElementNotVisibleException:
            return(df)
    return(df)

# Getting the stats for visits

In [35]:
def get_ninetyday_views(driver):
    select = driver.find_elements_by_css_selector
    select_one = driver.find_element_by_css_selector
    #go to profile page
    profile = select_one('li#profile-nav-item')
    profile.click()
    dropdown = select_one('ul#nav-settings__dropdown-options')
    url = dropdown.find_element_by_css_selector('a')
    url.get_attribute('href')
    driver.get(url.get_attribute('href'))

    #get the profile view information
    try:
        views = select_one('a.profile-views')
        views.get_attribute('href')
        driver.get(views.get_attribute('href'))
        views = select_one('span.me-wvmp-views__90-days-views')
        ninety_day_views = views.text

    except NoSuchElementException:
        try:
            ninety_day_views = select_one('span.pv-dashboard-section__metric-count').text
        except NoSuchElementException:
            ninety_day_views = 'unable to get'
    #number of views in the last 90 days
    return(ninety_day_views)

In [30]:
def get_ssindex(driver):
    select = driver.find_elements_by_css_selector
    select_one = driver.find_element_by_css_selector
    social_index_df = pd.DataFrame(columns=['ninety_day_views','social_score','weekly_totals'])
    driver.get('https://www.linkedin.com/')
    views = get_ninetyday_views(driver)
    #accessing your social selling index: Must be logged in then go to https://www.linkedin.com/sales/ssi
    driver.get('https://www.linkedin.com/sales/ssi')
    body = select_one('article#main')
    wrappers = body.find_element_by_css_selector('div#current-barchart-wrapper')
    score_list = wrappers.text.split('\n')
    labels = score_list[4:8]
    scores = score_list[-4:]
    #put this into the dataframe
    scores_tuple_list = list(zip(labels,scores))
    weekly_totals = select_one('div#overtime-linechart-accessible')
    time.sleep(5)
    table = weekly_totals.find_element_by_css_selector('table')
    body = table.find_element_by_css_selector('tbody')
    data = body.find_elements_by_css_selector('td')
    data_table_list = []
    for d in data:
        data_table_list.append(d.get_attribute('innerHTML'))
    #put this into the dataframe
    social_index = social_index_df.append({'ninety_day_views': views,
                    'social_score': scores_tuple_list,
                    'weekly_totals': data_table_list}, 
                    ignore_index=True)
    return(social_index)

In [43]:
def run_scrape(login, password):
    chromedriver = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(chromedriver)
    sign_in(login, password,driver)
    #messages = get_messages(driver)
    messages = get_messages(driver)
    #for saving to csv
    filename = f'{login}'+'.messages.csv'
    messages.to_csv(filename,index=False)
    ssindex = get_ssindex(driver)
    socialfilename = f'{login}'+'social_score.csv'
    ssindex.to_csv(socialfilename,index=False)
    
    

In [69]:
chromedriver = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chromedriver)
driver = sign_in(login, password,driver)

In [71]:
select = driver.find_elements_by_css_selector
select_one = driver.find_element_by_css_selector
df = pd.DataFrame(columns=['id','message','date'])
    #accessing the messages once logged in
message_btn = select_one('span#messaging-tab-icon')
message_btn.click()

In [72]:
unordered_list = select_one('ul.msg-conversations-container__conversations-list')
convo_list = unordered_list.find_elements_by_css_selector('a')
for c in convo_list:
    print(c.get_attribute('href'))

https://www.linkedin.com/messaging/thread/6495506015738568704/
https://www.linkedin.com/messaging/thread/6514721863699173376/
https://www.linkedin.com/messaging/thread/6504569339771969536/
https://www.linkedin.com/messaging/thread/6505188061725495296/
https://www.linkedin.com/messaging/thread/6495506053269192704/
https://www.linkedin.com/messaging/thread/6478373310269919233/
https://www.linkedin.com/messaging/thread/6458159242158444544/
https://www.linkedin.com/messaging/thread/6451525973451038720/
https://www.linkedin.com/messaging/thread/6444092850438307840/
https://www.linkedin.com/messaging/thread/6409963611464310784/
https://www.linkedin.com/messaging/thread/6427650795105320960/
https://www.linkedin.com/messaging/thread/6423406668243103744/
https://www.linkedin.com/messaging/thread/6422632323891687424/
https://www.linkedin.com/messaging/thread/6375431192233287681/


In [89]:
run_scrape(login, password)

0


In [90]:
!ls

geckodriver.log                   keri@multerer.comsocial_score.csv
keri@multerer.com.messages.csv    workbook.ipynb


In [91]:
df = pd.read_csv('keri@multerer.com.messages.csv')

In [92]:
df

,id,message,date


In [ ]:
!rm 'keri@multerer.com.messages.csv'
!rm 'keri@multerer.comsocial_score.csv'

In [ ]:
!ls